# Running ROUGE-Based Model
This Notebook serves to run our ROUGE-Based models.
All models can be found under `models`.

## Imports and libraries

In [1]:
import pandas as pd
import torch
from rouge_score import *
from ROUGE_Model_Loader import ROUGEModelLoader
from tqdm.notebook import tqdm

Below is our train data merged with prompts

In [2]:
data = pd.read_csv('./data/merged.csv')
data.head()

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,8a31b8cc1996,3b9047,In the social pyramid of ancient Egypt the pha...,-0.077267,0.424365,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
1,4387107feb4d,3b9047,The ancient Egyptian system of government was ...,1.376083,2.389443,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,3b784d0a5c8f,3b9047,Nobles were the only ont that could hold gover...,0.467722,-0.085653,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,1b2ff4d4edd9,3b9047,They were many different social classes. The p...,-0.012957,-0.409480,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,108049c01946,3b9047,The ancient Egyptian system of goverment is in...,2.204640,-0.645344,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...


## Selecting a Model

In [3]:
# Replace the parameters to specify the model
target_score = 'both'
hidden_dim = 64
epochs = 10
lr = 0.01

ROUGE_MODEL_PATH = f'./models/rouge_based_models/ROUGE_Based_Model_{target_score}_{hidden_dim}_{epochs}_{lr}.pt'

model_loader = ROUGEModelLoader(data, hidden_dim, target_score)
#print(model_loader.train(epochs, lr)) # load or train model
model = model_loader.model
model.load_state_dict(torch.load(ROUGE_MODEL_PATH)) # load or train model

<All keys matched successfully>

## Running the ROUGE-Based Model for a single data point

In [4]:
row = data.iloc[0]
output = model(row.text, row.prompt_text)
output

tensor([1.0046, 1.5386], grad_fn=<ViewBackward0>)

## Running the transformer on pandas frame
Adapt the code for your needs

In [5]:
content_preds = []
wording_preds = []

# Replace data for your own data. Here we use the first 5 rows from the merged.csv
for index, row in tqdm(data[:5].iterrows(), total=data[:5].shape[0]):
    outputs = model(row.text, row.prompt_text)

    if target_score in ('content', 'both'):
        content_preds.append(outputs[0].item())
    if target_score == 'both':
        wording_preds.append(outputs[1].item())
    elif target_score == 'wording':
        wording_preds.append(outputs[0].item())

preds = {}
if target_score in ('content', 'both'):
    preds |= {'content': content_preds}
if target_score in ('wording', 'both'):
    preds |= {'wording': wording_preds}

submission_df = pd.DataFrame(preds)
submission_df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,content,wording
0,1.004646,1.538599
1,1.181845,1.664008
2,1.097804,1.124133
3,1.030681,1.858219
4,1.469000,1.188908
